In [1]:
import os
import hickle as hkl
import numpy as np
import pybedtools
from pkg_resources import resource_filename
from torch import nn
import torch.nn.functional as F
import torch
from torch.utils.data import DataLoader, Dataset
from janggu.data import Bioseq, Cover, ReduceDim, SqueezeDim, Transpose
import torch.optim as optim
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

from IPython.display import Image

Using TensorFlow backend.


In [2]:
os.environ['JANGGU_OUTPUT'] = '/home/wangccy/janggu_out' 

In [3]:
order = 3

In [ ]:
!shuf -n 1 /home/wangccy/anaconda3/envs/final-proj/lib/python3.9/site-packages/janggu/roi_train.bed > /home/wangccy/anaconda3/envs/final-proj/lib/python3.9/site-packages/janggu/roi_train_rand1.bed
#set the random number of gene taking for training, you can change 1 to any number change file name in the output

In [ ]:
!shuf -n 1 /home/wangccy/anaconda3/envs/final-proj/lib/python3.9/site-packages/janggu/roi_test.bed > /home/wangccy/anaconda3/envs/final-proj/lib/python3.9/site-packages/janggu/roi_test_rand1.bed 
#set the random number of gene taking for testing, you can change 1 to any number change file name in the output

In [4]:
REFGENOME = resource_filename('janggu', 'hg19.fa')

#ROI_TRAIN_FILE = resource_filename('janggu', 'roi_train.bed')

ROI_TRAIN_FILE = resource_filename('janggu', 'roi_train_rand1.bed') # change file name here accordingly


ROI_TEST_FILE = resource_filename('janggu', 'roi_test_rand1.bed') # change file name here accordingly

PEAK_FILE = '/home/wangccy/sites_all.bed'


In [5]:
#Prepare Data
NN_DNA_train = Bioseq.create_from_refgenome('dna', refgenome=REFGENOME,
                                                       roi=ROI_TRAIN_FILE,
                                                       binsize=50,
                                                       order=3,
                                                       flank=150,
                                                       cache=False,
                                                       verbose=True)
NN_Labels_train = ReduceDim(Cover.create_from_bed('sites_all', roi=ROI_TRAIN_FILE,
                                                   bedfiles=PEAK_FILE,
                                                   binsize=50,
                                                   resolution=1,
                                                   flank=150,
                                                   mode='name_category',
                                                   conditions=['start']))

NN_DNA_test = Bioseq.create_from_refgenome('dna', refgenome=REFGENOME,
                                                       roi=ROI_TEST_FILE,
                                                       binsize=50,
                                                       order=3,
                                                       flank=150,
                                                       cache=False,
                                                       verbose=True)
NN_Labels_test = ReduceDim(Cover.create_from_bed('sites_all', roi=ROI_TEST_FILE,
                                                   bedfiles=PEAK_FILE,
                                                   binsize=50,
                                                   resolution=1,
                                                   flank=150,
                                                   mode='name_category',
                                                   conditions=['start']))

In [6]:
#Define DNAConvNet model
class DNAConvNet(nn.Module):
    def __init__(self, order=1):
        super(DNAConvNet, self).__init__()
        self.conv1 = nn.Conv1d(pow(4, order), 30, 21)
        self.dense = nn.Linear(30, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        
        # emulates global_max_pooling
        x = F.max_pool1d(x, x.size()[-1]).flatten(1, -1)
        x = self.dense(x)
        x = self.sigmoid(x)
        return x
    
net = DNAConvNet(3)
print(net)

DNAConvNet(
  (conv1): Conv1d(64, 30, kernel_size=(21,), stride=(1,))
  (dense): Linear(in_features=30, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [7]:
# Loading the data
class ToTensor(torch.utils.data.Dataset):
    def __init__(self, data, dtype='float32'):
        self.data = data
        self.dtype = dtype
        
    def __len__(self):
        return len(self.data)
    
    def __repr__(self):
        return "ToTensor({})".format(str(self.data))
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        data = self.data[idx].astype(self.dtype)
        
        return torch.from_numpy(data)

In [8]:
DNA_ = ToTensor(Transpose(ReduceDim(NN_DNA_train, axis=2), axis=(0,2,1)), dtype='float32')
DNA_

ToTensor(SqueezeDim(ReduceDim(Bioseq("dna"))))

In [9]:
class InOutTuple(Dataset):
    def __init__(self, inputs, labels, batch_size=32, shuffle=True):
        self.inputs = inputs
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.ridx = np.random.permutation(len(inputs))
        
    def __len__(self):
        return int(np.ceil(len(self.inputs) / self.batch_size))
    
    def __getitem__(self, idx):
        
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        ridx = self.ridx[self.batch_size*idx:(self.batch_size*(idx+1))].tolist()
        # enforce type compatibility
        return self.inputs[ridx], self.labels[ridx]

In [10]:
IODATA = InOutTuple(DNA_, ToTensor(NN_Labels_train))
len(IODATA)

1626

In [11]:
dataloader = DataLoader(IODATA, batch_size=1,
                        shuffle=False, num_workers=2)

In [12]:
#Training
criterion = nn.BCELoss()
optimizer = optim.Adadelta(net.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

net = net.to(device)

final_loss = 0.
for epoch in tqdm(range(100)):
    for i, d in enumerate(dataloader, 0):
        inputs, labels = d
        # when using dataloaders a dummy dimension
        # is introduced which we don't need.
        # with view we eliminate it.
        inputs = inputs.view(inputs.shape[1:])
        labels = labels.view(labels.shape[1:])
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        
        optimizer.step()
        final_loss = loss.item()

print('#' * 40)
print('loss: {}'.format(final_loss))
print('#' * 40)

cuda


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [13:22<00:00,  8.03s/it]

########################################
loss: 0.0
########################################


In [13]:
# Testing the model

DNA_TEST_ = ToTensor(Transpose(ReduceDim(NN_DNA_test, axis=2), axis=(0,2,1)), dtype='float32')

DNA_TEST_

ToTensor(SqueezeDim(ReduceDim(Bioseq("dna"))))

In [14]:
net.to('cpu')

DNAConvNet(
  (conv1): Conv1d(64, 30, kernel_size=(21,), stride=(1,))
  (dense): Linear(in_features=30, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [15]:
pred = net(DNA_TEST_[:]).detach().numpy()

In [16]:
print("AUC:", roc_auc_score(NN_Labels_test[:], pred))

AUC: 0.7040475230129783
